In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from collections import OrderedDict

### Importando tabelas

In [14]:
df_matriculas = pd.DataFrame()

# importando notas de 2010 a 2017
for i in range(10,18):
    for j in range(1,3):
        print('20'+str(i)+str(j))
        df_matricula = pd.read_csv('data_science/matriculas_new/matricula-componente-20'+str(i)+str(j)+'.csv',sep=';')
        #concatena as tabelas de 2010 a 2017
        df_matriculas = pd.concat([df_matriculas, df_matricula])

# obtendo o id do curso de ciências e tecnologia
df_cursos = pd.read_csv('data_science/cursos-graduacao.csv', sep=';')
id_cet = df_cursos[df_cursos.nome == 'CIÊNCIAS E TECNOLOGIA'].id_curso
id_cet

20101
20102
20111
20112
20121
20122
20131
20132
20141
20142
20151
20152
20161
20162
20171
20172


17    10320810
Name: id_curso, dtype: int64

In [37]:
df_turmas = pd.read_csv('data_science/turmas-2010-2017-ok.csv')
df_turmasFilt = df_turmas[(df_turmas['ano'] >= 2014) & (df_turmas['nivel_ensino'] == 'GRADUAÇÃO')]
df_turmasFilt.ano.value_counts()

/home/andreza/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2014    22531
2015    22467
2017    22309
2016    21946
Name: ano, dtype: int64

In [38]:
df_componentes = pd.read_csv('data_science/componentes-curriculares-presenciais-ok.csv')

In [39]:
print(df_turmasFilt.columns)
print(df_componentes.columns)

Index(['Unnamed: 0', 'id_turma', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes'],
      dtype='object')
Index(['Unnamed: 0', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_pro

### Merges

In [40]:
df_turmasNome = pd.merge(df_turmasFilt, df_componentes, left_on='id_componente_curricular', right_on='id_componente')
df_turmasNome.columns

Index(['Unnamed: 0_x', 'id_turma', 'codigo_turma', 'id_docente_interno',
       'id_docente_externo', 'observacao', 'id_componente_curricular',
       'nivel_ensino', 'campus_turma', 'local', 'ano', 'periodo',
       'data_inicio', 'data_fim', 'descricao_horario', 'total_solicitacoes',
       'capacidade_aluno', 'tipo', 'distancia', 'data_consolidacao',
       'agrupadora', 'id_turma_agrupadora', 'qtd_aulas_lancadas',
       'situacao_turma', 'convenio', 'modalidade_participantes',
       'Unnamed: 0_y', 'id_componente', 'tipo_componente', 'codigo', 'nivel',
       'nome', 'unidade_responsavel', 'ch_teorico', 'ch_pratico', 'ch_estagio',
       'ch_total', 'ch_dedicada_docente', 'ch_ead', 'cr_max_ead',
       'permite_horario_flexivel', 'qtd_unidades', 'procedimentos_avaliacao',
       'equivalencia', 'pre_requisito', 'co_requisito', 'ementa',
       'bibliografia', 'objetivos', 'conteudo', 'competencias_habilidades',
       'referencias', 'ano_programa', 'periodo_programa', 'modalidade

In [117]:
disciplinas = ['CÁLCULO I','CÁLCULO II','CÁLCULO III','VETORES E GEOMETRIA ANALÍTICA','PRÉ-CÁLCULO','PROBABILIDADE E ESTATÍSTICA',
               'INTRODUÇÃO À FÍSICA CLÁSSICA I','INTRODUÇÃO À FÍSICA CLÁSSICA II','INTRODUÇÃO À FÍSICA CLÁSSICA III','LÓGICA DE PROGRAMAÇÃO',
               'LINGUAGEM DE PROGRAMAÇÃO','COMPUTAÇÃO NUMÉRICA','QUÍMICA GERAL','ALGEBRA LINEAR','MECÂNICA DOS SÓLIDOS','MECÂNICA DOS FLUIDOS',
               'CIÊNCIA E TECNOLOGIA DOS MATERIAIS']

In [118]:
df_auxTurmas = df_turmasNome[df_turmasNome['nome'].isin(disciplinas)]

In [119]:
colunas = ['id_turma', 'codigo', 'nome', 'pre_requisito']
df_auxTurmas = df_auxTurmas[colunas]

df_auxAll = pd.merge(df_matriculas, df_auxTurmas, on='id_turma')

colunas2 = ['descricao', 'discente', 'id_turma', 'media_final', 'codigo', 'nome']
df_auxAll = df_auxAll[colunas2].drop_duplicates().dropna()

In [120]:
df_auxAll.head()

,descricao,discente,id_turma,media_final,codigo,nome
2,REPROVADO,b70cfc26a1b6eb885a529e60c07354cd,1226773,3.4,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
16,REPROVADO,970cbdd2d4d5d75cc06acf545766eba1,1226773,0.0,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
22,REPROVADO,7ba56f4d760d44531b3f8220c58e947b,1226773,0.0,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
52,APROVADO,dfdf0394ccc40abbcbc8aa6ec6e49969,1220717,5.4,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS
58,REPROVADO,db1fd7f22172df2305735834721d934a,1220717,4.7,ECT1401,CIÊNCIA E TECNOLOGIA DOS MATERIAIS


In [121]:
df_auxAll.nome.value_counts()

CÁLCULO I                             6912
VETORES E GEOMETRIA ANALÍTICA         5779
PRÉ-CÁLCULO                           4276
PROBABILIDADE E ESTATÍSTICA           4129
CÁLCULO II                            3928
QUÍMICA GERAL                         3874
LINGUAGEM DE PROGRAMAÇÃO              3859
COMPUTAÇÃO NUMÉRICA                   3229
CÁLCULO III                           3216
MECÂNICA DOS SÓLIDOS                  2969
CIÊNCIA E TECNOLOGIA DOS MATERIAIS    2962
INTRODUÇÃO À FÍSICA CLÁSSICA I        2918
LÓGICA DE PROGRAMAÇÃO                 2762
INTRODUÇÃO À FÍSICA CLÁSSICA II       2512
MECÂNICA DOS FLUIDOS                  2483
INTRODUÇÃO À FÍSICA CLÁSSICA III      1882
ALGEBRA LINEAR                         514
Name: nome, dtype: int64

### Correlação

In [124]:
# apenas os aprovados
df_auxAprovados = df_auxAll[df_auxAll['descricao'].str.contains('APROVADO')]

In [125]:
df_auxAprovados.descricao.value_counts()

APROVADO POR NOTA    18193
APROVADO              9587
Name: descricao, dtype: int64

In [64]:
data = OrderedDict(
{
'País': ['Bélgica', 'Índia', 'Brasil','Índia'],
'Capital': ['Bruxelas', 'Nova Delhi', 'Brasília', 'Nova Delhi'],
'População': [123465, 456789, 987654, 456789]
})

df = pd.DataFrame(data)
df

,País,Capital,População
0,Bélgica,Bruxelas,123465
1,Índia,Nova Delhi,456789
2,Brasil,Brasília,987654
3,Índia,Nova Delhi,456789


In [128]:
discentes= df_auxAprovados.discente.unique()

In [156]:
isso = df_auxAll[(df_auxAll['nome']=='CÁLCULO I') & (df_auxAll['discente']=='fe16b848a14a942f60497ffaf3bde2f9')].media_final
isso.

In [145]:
testes = pd.DataFrame()

for discente in discentes:
    calculo1 = df_auxAprovados[(df_auxAprovados['nome']=='CÁLCULO I') & (df_auxAprovados['discente']==discente)].media_final
    calculo2 = df_auxAprovados[(df_auxAprovados['nome']=='CÁLCULO II') & (df_auxAprovados['discente']==discente)].media_final
    teste = [discente,
             calculo1.values,
             calculo2.values]
#     testes = pd.concat([testes, teste])
    print(teste)

['dfdf0394ccc40abbcbc8aa6ec6e49969', array([], dtype=float64), array([], dtype=float64)]
['033119b099e8f653edeac0b9dac28919', array([], dtype=float64), array([5.])]
['592da2a6c2fb0edb08e66ccb52960864', array([], dtype=float64), array([], dtype=float64)]
['09ad39ab04e9ab4df7eb8f4b45395c83', array([], dtype=float64), array([], dtype=float64)]
['e4875a0e46e6a5956f4e70b1344e85a2', array([], dtype=float64), array([], dtype=float64)]
['972e8501dba691a5c983573f98997116', array([], dtype=float64), array([], dtype=float64)]
['39ff0846e2124fecd04e324ed9f49dca', array([], dtype=float64), array([], dtype=float64)]
['5e1729c775651cd071fb50d3f69552bd', array([], dtype=float64), array([], dtype=float64)]
['7ef6312532aae6c5d6f914954c3c05a0', array([6.5]), array([], dtype=float64)]
['dd55926736820cdb74384e849914d6b4', array([], dtype=float64), array([], dtype=float64)]
['f473d733c65120e70853433f2ce644cd', array([], dtype=float64), array([], dtype=float64)]
['2817bf417a611cefb2f2e5b9e39f4e0b', array([],